In [79]:
#  Седьмая (и последняя) лаба будет посвящена нейронным сетям. 
#     Для этого нужно взять датасет MNIST и сделать однослойный\многослойный перцептрон для распознавания цифр 0-9. 
#     При этом обязательно, чтобы можно было посмотреть на каких цифрах ваш классификатор ошибается, 
#     а так же должна быть возможность ручного ввода изображений для распознавания и дообучения на них. 
#     Нужно грамотно уметь объяснять все про backpropagation и функции активации. Датасет и инструкция, 
#     как из него извлекать данные лежат здесь: http://yann.lecun.com/exdb/mnist/index.html 

# Описание алгоитма: https://habrahabr.ru/post/198268/

In [21]:
%matplotlib inline
%load_ext autoreload
%autoreload 5
%autosave 15    

from numpy import append, array, int8, uint8, zeros
import pandas as pd
import random as rd
import pylab as pl
import matplotlib.pyplot as plt
import os, struct
from math import exp
from array import array as pyarray
from sklearn.cross_validation import KFold
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.metrics import accuracy_score, f1_score
from scipy.spatial.distance import euclidean, cityblock, cosine, correlation
from IPython.display import display, HTML
from sklearn.metrics import mean_squared_error
plt.rcParams["figure.figsize"]=[15,10]

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


Autosaving every 15 seconds


In [80]:
from mnist import MNIST

mndata = MNIST('./samples')

# size 28 * 28, train count = 60000, test count = 10000
image_size = 28 # n = 28 ** 2

train_images, train_labels = mndata.load_training()
train_size = len(train_images)

test_images, test_labels = mndata.load_testing()
test_size = len(test_images)


In [81]:
# init 


def sumprod(a, b):
    return sum(map(lambda x: x[0] * x[1], zip(a, b)))

n = image_size ** 2
m = 10 # количество выходных y нейронов
p = 50 # TODO количество скрытых z нейронов


v = [[rd.random() - 0.5 for j in range(p)] for i in range(n)] # Смещение скрытого нейрона j
v0 = [rd.random() - 0.5 for j in range(p)]
w = [[rd.random() - 0.5 for k in range(m)] for j in range(p)] # Смещение нейрона на выходе
w0 = [rd.random() - 0.5 for k in range(m)]

alpha = 0.5 # Скорость обучения

In [82]:
# функция активации (competitive function)

f = lambda x: 1 / (1 + exp(-x)) # [0..1]
f1 = lambda x: f(x) * (1 - f(x)) # z1'(x)

In [83]:
# neural network

def nn_go(x, ans):
    global v, w, v0, w0
    t = [0] * m
    t[ans] = 1
    xx = list(map(lambda x: x / 256.0, x))
    # step 4. evaluate z
    z_in = [0] * p # Суммарное значение подаваемое на вход скрытого нейрона
    for j in range(p):
        z_in[j] = sumprod([v[i][j] for i in range(n)], xx) + v0[j]
    zz = list(map(f, z_in)) # Скрытый нейрон
    
    # step 5. evaluate y
    y_in = [0] * m # Суммарное значение подаваемое на вход скрытого нейрона
    for k in range(m):
        y_in[k] = sumprod([w[j][k] for j in range(p)], zz) + w0[k]
    yy = list(map(f, y_in)) # Нейрон на выходе    
    
    if (ans == -1):
        res = max(yy)
        for k in range(m):
            if yy[k] == res:
                return k
    
    # step 6. evaluate y error
    sigmak = [(t[k] - yy[k]) * (f1(y_in[k])) for k in range(m)]
    delta_w = [[alpha * sigmak[k] * zz[j] for k in range(m)] for j in range(p)]
    delta_w0 = [alpha * sigmak[k] for k in range(m)]
    
    # step 7. evaluate z error
    sigma_in = [sumprod(sigmak, w[j]) for j in range(p)]
    sigmaj = [sigma_in[j] * f1(z_in[j]) for j in range(p)]
    delta_v = [([alpha * sigmaj[j] * xx[i] for j in range(p)]) for i in range(n)] # TODO
    delta_v0 = [alpha * sigmaj[j] for j in range(p)]
    
    # change weights
    v = [[v[i][j] + delta_v[i][j] for j in range(p)] for i in range(n)] # Смещение скрытого нейрона j
    v0 = [v0[j] + delta_v0[j] for j in range(p)]
    w = [[w[j][k] + delta_w[j][k] for k in range(m)] for j in range(p)] # Смещение нейрона на выходе
    w0 = [w0[k] + delta_w0[k] for k in range(m)]
    
    return mean_squared_error(yy, t)

In [86]:
# neural learning
for steps in range(10):
    for num in range(100): #train_size    
        nn_go(train_images[num], train_labels[num])    

KeyboardInterrupt: 

In [87]:
def getNumber(x):
    res = nn_go(x, -1)    
    return res

# neural test
ok = 0
failed = 0
for i in range(100): # test_size
    ans = test_labels[i]
    pred = getNumber(test_images[i])
#     print("answer is", ans, "; prediction is", pred)
    if (ans == pred):
        ok += 1
    else:
        failed += 1
print("OK =", ok, "; FAILED =", failed)

OK = 67 ; FAILED = 33
